# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## -----------------------------Third Part of the assignment (week 3)-----------------------------

### Explore and cluster the neighborhoods in Toronto

In [5]:
# import libraries

import matplotlib.cm as cm
import matplotlib.colors as colors

import pandas as pd

!pip install -U geopy
from sklearn.cluster import KMeans 
from sklearn.datasets import make_blobs
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

import json # library to handle JSON files
import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: geopy in /home/tbfk/.local/lib/python3.8/site-packages (2.0.0)


#### Load data from previous part of the assignment

In [6]:
# read csv file
df_toronto = pd.read_csv('applied_DS_week3_part2.csv')
print('done loading the csv file')

done loading the csv file


In [7]:
df_toronto = df_toronto.drop(df_toronto.columns[0], axis=1) #drop first unnamed column
df_toronto.head(20)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Create map of Toronto

#### Get coordinates of Toronto

In [8]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Plot map of Toronto with neighbourhoods from dataset

In [9]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Segment and cluster only the neighbourhoods of North York

#### Get data with neighbourhoods from borough North York

In [10]:
north_york_data = df_toronto[df_toronto['Borough'] == 'North York'].reset_index(drop=True)
north_york_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714
4,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493


#### Get geographical coordinates of North York

In [11]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


#### Visualize the neighbourhoods of North York

In [12]:
# create map of North York using latitude and longitude values
map_north_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(north_york_data['Latitude'], north_york_data['Longitude'], north_york_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_north_york)  
    
map_north_york

### Get Venues in North York

#### Define Foursquare Credentials and Version

In [13]:
CLIENT_ID = '01OVT5KIQSCY5QZDIOSYKWG4CS5TKJMOLVXFZVLJIHEXYI44' # your Foursquare ID
CLIENT_SECRET = '0QBZNZLQDCPTDNKSIM051QCREPEBH5ONLINXV15NC3F10F2I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 01OVT5KIQSCY5QZDIOSYKWG4CS5TKJMOLVXFZVLJIHEXYI44
CLIENT_SECRET:0QBZNZLQDCPTDNKSIM051QCREPEBH5ONLINXV15NC3F10F2I


#### Second neighbourhood of north york

In [14]:
north_york_data.loc[1, 'Neighbourhood']

'Fairview, Henry Farm, Oriole'

#### Get Hillcrest Village's latitude and longitude values.

In [15]:
neighborhood_latitude = north_york_data.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = north_york_data.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = north_york_data.loc[1, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Fairview, Henry Farm, Oriole are 43.7785175, -79.3465557.


#### Get top 100 Venues in Fairview

In [16]:
search_query = north_york_data.loc[1, 'Neighbourhood']
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=01OVT5KIQSCY5QZDIOSYKWG4CS5TKJMOLVXFZVLJIHEXYI44&client_secret=0QBZNZLQDCPTDNKSIM051QCREPEBH5ONLINXV15NC3F10F2I&ll=43.7785175,-79.3465557&v=20180605&query=Fairview, Henry Farm, Oriole&radius=500&limit=100'

#### Send the GET request

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa9ced423e28d6ac0ce5efc'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'query': 'fairview henry farm oriole',
  'totalResults': 11,
  'suggestedBounds': {'ne': {'lat': 43.7830175045, 'lng': -79.34033481358881},
   'sw': {'lat': 43.7740174955, 'lng': -79.35277658641118}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ae07b8bf964a5208c7f21e3',
       'name': 'Apple Fairview',
       'location': {'address': '1800 Sheppard Ave E',
        'crossStreet': 'ar Fairview Mall Dr',
        'lat': 43.7778832,
        'lng': -79.3437891,
        'labeledLatLngs': [

#### Extract Venues in Fairview

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Apple Fairview,Electronics Store,43.777883,-79.343789
1,CF Fairview Mall,Shopping Mall,43.777994,-79.343665
2,New York Fries - Fairview Mall,Restaurant,43.778605,-79.343577
3,Fairview Mall Food Garden,Food Court,43.778369,-79.343216
4,Fairview Library Theatre,Theater,43.779018,-79.346526


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

11 venues were returned by Foursquare.


### Explore Neighborhoods in North York

#### Function to get all the Venues from all the neighboorhoods in North York 

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
north_york_venues = getNearbyVenues(names=north_york_data['Neighbourhood'],
                                   latitudes=north_york_data['Latitude'],
                                   longitudes=north_york_data['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Manor, Lawrence Heights
Glencairn
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Humberlea, Emery


In [23]:
print(north_york_venues.shape)
north_york_venues.head()

(240, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,Hillcrest Village,43.803762,-79.363452,A.Y. Jackson Secondary School Track,43.805068,-79.366677,Athletics & Sports


#### Count the number of venues for each neighbourhood

In [24]:
north_york_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Don Mills,23,23,23,23,23,23
Downsview,16,16,16,16,16,16
"Fairview, Henry Farm, Oriole",70,70,70,70,70,70
Glencairn,5,5,5,5,5,5
Hillcrest Village,5,5,5,5,5,5
Humber Summit,2,2,2,2,2,2


#### Get number of unique venue categories

In [25]:
print('There are {} uniques categories.'.format(len(north_york_venues['Venue Category'].unique())))

There are 102 uniques categories.


### Analyze Each Neighbourhood

In [26]:
# one hot encoding
north_york_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_york_onehot['Neighbourhood'] = north_york_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [north_york_onehot.columns[-1]] + list(north_york_onehot.columns[:-1])
north_york_onehot = north_york_onehot[fixed_columns]

north_york_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
north_york_onehot.shape

(240, 103)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
north_york_grouped = north_york_onehot.groupby('Neighbourhood').mean().reset_index()
north_york_grouped

,Neighbourhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.095238,...,0.047619,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.0000,0.045455,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000
3,Don Mills,0.000000,0.0000,0.000000,0.043478,0.000000,0.043478,0.0000,0.000000,0.000000,...,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Downsview,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.062500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Fairview, Henry Farm, Oriole",0.000000,0.0000,0.014286,0.000000,0.000000,0.014286,0.0000,0.028571,0.028571,...,0.000000,0.014286,0.000000,0.014286,0.000000,0.014286,0.014286,0.014286,0.000000,0.042857
6,Glencairn,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.200000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Hillcrest Village,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.2000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Humber Summit,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Humberlea, Emery",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [29]:
north_york_grouped.shape

(19, 103)

#### Let's print each neighborhood along with the top 5 most common venues

In [30]:
num_top_venues = 5

for hood in north_york_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = north_york_grouped[north_york_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0     Coffee Shop  0.10
1            Bank  0.10
2  Ice Cream Shop  0.05
3     Bridal Shop  0.05
4   Deli / Bodega  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.09
1  Italian Restaurant  0.09
2         Coffee Shop  0.09
3       Grocery Store  0.05
4    Greek Restaurant  0.05


----Don Mills----
                 venue  freq
0                  Gym  0.13
1          Coffee Shop  0.09
2  Japanese Restaurant  0.09
3           Beer Store  0.09
4   Dim Sum Restaurant  0.04


----Downsview----
            venue  freq
0   Grocery Store  0.19
1            Park  0.12
2  Baseball Field  0.06
3           Hotel  0.06
4         Airport  0.06


----Fairview, Henry Farm, Or

#### Put it into a pandas dataframe

In [31]:
# sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each neighborhood

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = north_york_grouped['Neighbourhood']

for ind in np.arange(north_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Mobile Phone Shop,Sandwich Place,Gas Station,Diner,Ice Cream Shop,Deli / Bodega,Chinese Restaurant,Middle Eastern Restaurant
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Greek Restaurant,Grocery Store,Indian Restaurant,Juice Bar,Liquor Store,Locksmith,Comfort Food Restaurant
3,Don Mills,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Caribbean Restaurant,Clothing Store,Chinese Restaurant,Café,Dim Sum Restaurant,Italian Restaurant
4,Downsview,Grocery Store,Park,Bank,Liquor Store,Hotel,Shopping Mall,Home Service,Baseball Field,Business Service,Gym / Fitness Center


### Cluster Neighborhoods

#### Run k-means to cluster the neighborhood intro 4 clusters

In [33]:
# set number of clusters
kclusters = 4

north_york_grouped_clustering = north_york_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int32)

#### Create new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

north_york_merged = north_york_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
north_york_merged = north_york_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

north_york_merged = north_york_merged.dropna()

north_york_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0.0,Dog Run,Athletics & Sports,Mediterranean Restaurant,Pool,Golf Course,Fried Chicken Joint,Food Truck,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Juice Bar,Japanese Restaurant,Women's Store,Food Court,Shoe Store,Bakery,Bank
2,M2K,North York,Bayview Village,43.786947,-79.385975,0.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714,2.0,Martial Arts School,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
5,M2N,North York,"Willowdale, Willowdale East",43.770120,-79.408493,0.0,Ramen Restaurant,Japanese Restaurant,Sushi Restaurant,Sandwich Place,Café,Coffee Shop,Pizza Place,Hotel,Ice Cream Shop,Grocery Store


### Visualize the clusters

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_york_merged['Latitude'], north_york_merged['Longitude'], north_york_merged['Neighbourhood'], north_york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters